# PPO Breakout Example

### Atari Breakout

Please do note that this example may take a long time to train.

With the default 4 threads runnning on an 8-core CPU with a GTX 1080 Ti, it will take several hours to train to a decent level of play.

Running on a platform with more GPU power and a larger cluster of CPUs could siginificantly reduce training time.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten
from ludus.policies import PPOTrainer
from ludus.env import EnvController
from ludus.utils import preprocess_atari
import gym
# Super Mario stuff
from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [2]:
def make_env():
    env = gym_super_mario_bros.make('SuperMarioBros-v0')
    env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)
    return env

In [3]:
env = make_env() # This instance of the environment is only used
                              # to get action dimensions
in_shape = [84, 84, 1] # Size of reshaped observations

# Creating a conv net for the policy and value estimator
obs_op = Input(shape=in_shape)
conv1 = Conv2D(16, 8, (4, 4), activation='relu')(obs_op)
max_pool1 = MaxPool2D(2, 2)(conv1)
conv2 = Conv2D(32, 4, (2, 2), activation='relu')(max_pool1)
max_pool2 = MaxPool2D(2, 2)(conv2)
dense1 = Dense(256, activation='relu')(max_pool2)
flattened = Flatten()(dense1)

# Output probability distribution over possible actions
act_probs_op = Dense(env.action_space.n, activation='softmax')(flattened)

# Output value of observed state
value_op = Dense(1)(flattened)

# Wrap a Proximal Policy Optimization Trainer on top of the network
network = PPOTrainer(obs_op, act_probs_op, value_op, act_type='discrete', ppo_iters=80)

/home/ejmejm/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
n_episodes = 10000 # Total episodes of data to collect
max_steps = 2048 # Max number of frames per game
batch_size = 4 # Smaller = faster, larger = stabler
print_freq = 1 # How many training updates between printing progress

In [5]:
# Create the environment controller for generating game data
ec = EnvController(make_env, n_threads=4)
# Set the preprocessing function for observations
ec.set_obs_transform(lambda x: preprocess_atari(x.squeeze()))

In [7]:
update_rewards = []

for i in range(int(n_episodes / batch_size)):
    ec.sim_episodes(network, batch_size, max_steps) # Simualate env to generate data
    update_rewards.append(ec.get_avg_reward()) # Append rewards to reward tracker list
    dat = ec.get_data() # Get all the data gathered
    network.train(dat) # Train the network with PPO
    if i != 0 and i % print_freq == 0:
        print(f'Update #{i}, Avg Reward: {np.mean(update_rewards[-print_freq:])}') # Print an update

Update #1, Avg Reward: 1822.0
Update #2, Avg Reward: 1698.0
Update #3, Avg Reward: 1473.5
Update #4, Avg Reward: 1478.0
Update #5, Avg Reward: 1624.5
Update #6, Avg Reward: 1857.0
Update #7, Avg Reward: 1498.0
Update #8, Avg Reward: 1369.75
Update #9, Avg Reward: 1884.0
Update #10, Avg Reward: 1176.5
Update #11, Avg Reward: 1600.25
Update #12, Avg Reward: 1208.5
Update #13, Avg Reward: 1287.5
Update #14, Avg Reward: 964.25
Update #15, Avg Reward: -313.0
Update #16, Avg Reward: -630.0
Update #17, Avg Reward: 145.75
Update #18, Avg Reward: 694.0
Update #19, Avg Reward: 481.0
Update #20, Avg Reward: 744.0
Update #21, Avg Reward: 815.75
Update #22, Avg Reward: 746.5


KeyboardInterrupt: 

In [ ]:
ec.render_episodes(network, 5, max_steps) # Render an episode to see the result